In [1]:
import os
import pandas as pd
import numpy as np
import data_wrangle
import json
from importlib import reload
import preprocess
import sqlite3

import prettytable as pt


#From PCA
from sklearn.decomposition import PCA
from sklearn import decomposition
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import scipy.signal as ss
import pybaselines.spline as py

import pandas as pd
import numpy as np
from math import ceil


# import plotly.io as pio
# pio.renderers.default = "browser"
# pio.renderers.default = "png"



In [2]:
#Detect Oversaturation of spectra
def oversat_check(spectrum, threshold=0.1, window=5):
    """
    Detects oversaturation in a given spectrum by checking if there is a section 
    where the slope of the spectrum is consistently below a given threshold 
    over a given window of points.

    Parameters
    ----------
    spectrum : tuple
        A tuple containing the x and y data of the spectrum to be analyzed.
    threshold : float, optional
        The threshold slope value below which a section of the spectrum is considered 
        to be oversaturated. Default is 0.1.
    window : int, optional
        The size of the window over which to calculate the slopes. Default is 5.

    Returns
    -------
    bool
        True if an oversaturated section is detected, False otherwise.
    """
    x,y = spectrum #Grab x,y data from tuple input
    
    slopes = np.diff(y) / np.diff(x)
    for i in range(len(slopes) - window + 1):
        if np.allclose(slopes[i:i+window], slopes[i], atol=threshold):
            return True
    return False

def unique_count(lst):
    result={}
    for item in lst:
        if item in result:
            result[item] += 1
        else:
            result[item]=1
    return result

In [110]:
path='/home/dunn/DBM/Brain.db'

In [32]:
reload(data_wrangle)
db=data_wrangle.DataTable('/mnt/c/Users/16162/Desktop/Brain.db','sheep')
db.dead_list.append(53)
db.dead_list.append(54)
db.dead_list.append(55)
db.dead_list.append(134) #bad spectra?

db.raman_shifts()
# db.condition='WHERE location IS NOT NULL'
db.condition='WHERE sheep IS 2'
x=db.raman_shifts()
ys=db.intens()

There are 5 found in the dataset.
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#code not working, featuring 'get_spec'

reload(data_wrangle)
db=data_wrangle.DataTable('/mnt/c/Users/16162/Desktop/Brain.db','sheep')
db.raman_shifts()
data=db.get_spectra()

xs=[sum(x)/len(x) for x in zip(*[tup[1] for tup in data])]
xs=[int(x) for x in xs]

data_dict={}
# Loop through each tuple in mylist
for tup in data:
    col_name = str(tup[0])
    col_data = tup[2]
    # Add the list as values to the dictionary with the column name as key
    data_dict[col_name] = col_data

# Create the DataFrame from the dictionary
df = pd.DataFrame(data_dict)

# Set the column names to be the values in the second element of each tuple
df.columns = [str(val) for val in data[0][1]]

There are 5 found in the dataset.
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

ValueError: Length mismatch: Expected axis has 226 elements, new values have 647 elements

Testing some dataframe manipulation

In [ ]:
df=pd.DataFrame([[1,2,3,4,5],[2,3,4,5,6],[1,2,3,4,5],[5,6,7,8,9],[1,2,3,4,5],[5,6,7,8,9]])
print(df)
uniques=set(map(tuple,df.values))
uniques=list(uniques)
for u in uniques:
    mask=np.all(df==u, axis=1)
    print(mask)
print(df[mask])
print(df)


   0  1  2  3  4
0  1  2  3  4  5
1  2  3  4  5  6
2  1  2  3  4  5
3  5  6  7  8  9
4  1  2  3  4  5
5  5  6  7  8  9
0    False
1    False
2    False
3     True
4    False
5     True
dtype: bool
0     True
1    False
2     True
3    False
4     True
5    False
dtype: bool
0    False
1     True
2    False
3    False
4    False
5    False
dtype: bool
   0  1  2  3  4
1  2  3  4  5  6
   0  1  2  3  4
0  1  2  3  4  5
1  2  3  4  5  6
2  1  2  3  4  5
3  5  6  7  8  9
4  1  2  3  4  5
5  5  6  7  8  9


In [ ]:
#Update find/replace

#Reload library to impliment changes
reload(data_wrangle)

#Access dataBase and dataTable
conn=sqlite3.connect('/mnt/c/Users/16162/Desktop/Brain.db')
c=conn.cursor()

#Set search conditions
date='2023-04-20'

#Col to update
col_to_update='spike2'
search='spiked2'

#Find entries per search conditions
c.execute(f'SELECT id, {col_to_update}, filename FROM sheep WHERE date = ?',(date,))
# c.execute(f'SELECT id, {col_to_update}, filename FROM sheep WHERE date = ?',(date,))
rows=c.fetchall()

#Loop through each entry
for row in rows:
    if search in row[2]:
        new='yes'
        c.execute(f'UPDATE sheep SET {col_to_update} = ? WHERE id = ?',(new,row[0],))
    # elif 'spiked2' in row[2]:
        # new='yes'
    


        
    #     # new='yes'
    #     for s in row[2].split('_'):
    #         if search in s:
    #             new=s
    # #             new= s.split(search)[0]
    # #             # print(new)
    # else:
    #     new='0'
      
    # c.execute(f'UPDATE sheep SET {col_to_update} = ? WHERE id = ?',(new,row[0],))
conn.commit()
conn.close()




In [ ]:
#Reload library to impliment changes
reload(data_wrangle)

#Access dataBase and dataTable
db=data_wrangle.DataTable('/home/dunn/DBM/Brain.db','sheep')

numbs=list(range(1,53))
data=db.get_spectra()
print(data)
data=[(tup[0], tup[1], preprocess.process(tup[2])) for tup in data]
# overs=[oversat_check((d[1], d[2])) for d in data]
# print(overs.count(True))
    

# traces=[go.Scatter(x=d[1], y=d[2], name=d[0]) for d in data]
traces=[go.Scatter(x=d[1], y=d[2], name=d[0]) for d in data]


fig=go.Figure(traces)
# fig.show()

data

[]


[]

In [ ]:
reload(data_wrangle)

db=data_wrangle.DataTable('/home/dunn/DBM/Brain.db','data')
db.condition='WHERE location IS NOT NULL'

<h1> PCA

In [33]:
class multivar:
    def __init__(self,intens,rs,color_by,
                 names, #color_dict,
                 hover=None,symbol=None,
                 ncomp=10):
        self.intens=intens #df
        self.rs=rs #series
        
        self.color=list(color_by)
        self.names=list(names)
        self.ncomp=ncomp

        # self.color_dict=color_dict,

        if hover==None:
            self.hover=self.names
        else:
            self.hover=hover
        if symbol==None:
            self.symbol=self.color
        else:
            self.symbol=symbol
        
        self.rs_ax='Raman Shift (cm-1)'

    
    def label_gen(self,prefix,n):
        nums=list(range(1,n+1))
        res=['%s %s'%(prefix, num) for num in nums]
        return res
    
    def symbol_gen(self,len_needed):
        syms=['circle','square','diamond','cross','x','triangle-up','star-triangle-down','triangle-left','triangle-se','pentagon','hexagon2','star','diamond-tall']
        if len_needed <= len(syms):
            return syms[0:len_needed]
        else:
            times_to_mult=ceil((len_needed/len(syms)))
            rep_syms=syms*times_to_mult
            return rep_syms[0:len_needed]

    def pca_run(self):
        pca=decomposition.PCA(n_components=self.ncomp)
        
        #Apply PCA
        X=scale(self.intens)
        X=pca.fit_transform(X)

        #Get components
        pc_labels=self.label_gen('PC',self.ncomp)
        pcs=pca.components_.T*np.sqrt(pca.explained_variance_)
        pcs=pd.DataFrame(pcs,columns=pc_labels, index=self.names)

        #Get explained variance
        explained_list=[round(x*100,2) for x in pca.explained_variance_ratio_]
        explained={pc_labels[i]:explained_list[i] for i in range(self.ncomp)}

        #Get loadings
        loadings=pd.DataFrame(X,columns=pc_labels,index=self.rs)
        return pcs, loadings, explained
    
    def pxfig(self,df,x,y,color,color_dict=None):
        syms=self.symbol_gen(len(df))
        fig=px.scatter(df,x=x,y=y,color=color,
                    #    color_discrete_map=color_dict,
                    #    color_continuous_scale=px.colors.qualitative.T10,
                    #    range_color=(-18,10),
                    #    symbol_sequence=syms, symbol=self.symbol,
                        symbol_sequence=['square','circle','diamond','cross'],
                        symbol=self.color,
                       #symbol_sequence=syms, symbol=self.names, #For color blind friendly figure
                       template='simple_white',hover_name=self.hover)
        
        # fig.update_xaxes(range=[0.8,0.9])
        fig.update_traces(marker_size=10)
        return fig

    def pca_figs(self,x,y,color=None,color_dict=None):
        if color is None:
            color=self.names

        pcs,loadings,explained=self.pca_run()

        #PCA scatter plot
        fig1=self.pxfig(pcs,x,y,color)#,color_dict=color_dict)
        fig1.update_layout(title='%s vs %s'%(x,y),font_size=20)
        fig1.update_xaxes(title_text='%s (%s%s)'%(x,explained[x],'%'),
                          range=[0.8,1])
        fig1.update_yaxes(title_text='%s (%s%s)'%(y,explained[y],'%'),
                          range=[0,0.5])
        
        # save_name='pc_zoom_mix'
        # pio.write_image(fig,'/mnt/c/Users/16162/Desktop/%s.svg'%save_name,
        #         width=1600,
        #         height=800)
        
        #PCA Loadings
        fig2=px.line(loadings,x=loadings.index,y=x,
                    title='%s (%s%s)'%(x,explained[x],'%'), template='simple_white',
                    labels={x:'','index':self.rs_ax})
        #fig.show()
        fig3=px.line(loadings,x=loadings.index,y=y,
                    title='%s (%s%s)'%(y,explained[y],'%'),template='simple_white',
                    labels={y:'','index':self.rs_ax})
        #fig.show()
        return fig1,fig2,fig3
    

    def pca_3d(self, color=None, color_dict=None, sym=None, x='PC 3', y='PC 2', z='PC 1'):
        if color is None:
            color=self.color
        if sym is None:
            pass
        pcs,loadings,explained=self.pca_run()
        
        fig=px.scatter_3d(pcs,x=x,y=y,z=z, color=color,
                        color_discrete_map=color_dict,
                        # symbol=self.color, symbol_sequence=['square', 'circle', 'diamond', 'cross'],
                        # color_continuous_scale=px.colors.sequential.Jet, range_color=(0,-20),  #Jet,
                    #   color_continuous_midpoint=3,
                        hover_name=self.hover
                        )
        fig.update_layout(height=800)#, showlegend=False)#, template='simple_white')
        fig.update_traces()
    
        # fig.update_layout(scene=dict(xaxis_width=800))
        # fig.update_xaxes(showline=True, linewidth=5, linecolor='black')
        # fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
        # fig.update_zaxes(showline=True, linewidth=2, linecolor='black')
        # fig.write_html('/mnt/c/Users/16162/Desktop/3dpca.html')
        fig.show()

    def pca_fig(self,xpc,ypc):#,color_dict):
        #delete?
        figs=self.pca_figs(xpc,ypc,color=self.color)
        figs[1].show()
        figs[2].show()

    def pca_fig(self,figs):
        fig=make_subplots(rows=2,cols=2,specs=[[{'rowspan':2},{}],[None,{}]],
                          subplot_titles=[
                          figs[0].layout.title.text,figs[1].layout.title.text,figs[2].layout.title.text],
                          vertical_spacing=0.35)
        for trace in figs[0].data:
            fig.add_trace(trace,row=1,col=1)
            fig.update_xaxes(title_text=figs[0].layout.xaxis.title.text,row=1,col=1)
            fig.update_yaxes(title_text=figs[0].layout.yaxis.title.text,row=1,col=1)
        for trace in figs[1].data:
            fig.add_trace(trace,row=1,col=2)
        for trace in figs[2].data:
            fig.add_trace(trace,row=2,col=2)

        fig.update_xaxes(title_text=figs[1].layout.xaxis.title.text,row=1,col=2)
        fig.update_xaxes(title_text=figs[1].layout.xaxis.title.text,row=2,col=2)
        fig.update_xaxes(title_text=figs[0].layout.xaxis.title.text,row=1,col=1)
        fig.update_yaxes(title_text=figs[0].layout.yaxis.title.text,row=1,col=1)

        fig.update_layout(template='simple_white',
                          font_size=20,
                          font_family='ariel')
        return fig

<h1> Data Work

<h2>Get Data From DB

In [ ]:
reload(data_wrangle)
# db=data_wrangle.DataTable('/home/dunn/DBM/Brain.db','sheep') #Imported with tobytes
# db=data_wrangle.DataTable('/home/dunn/DBM/Brain.db','data')
# db.condition='WHERE strftime(\'%Y\',date)=\'2023\''
# db.condition='WHERE strftime(\'%Y\',date)=\'2023\' AND general_loc in (\'substantia nigra\',\'cortex\',\'midbrain\')'
# db.condition="WHERE date LIKE '2023-%'"# AND location IN ('cortex')"
# db.table_info()
x=db.raman_shifts()
# ys=db.intens()
namedic=db.names()
print(namedic)
ys=db.intens()

+---------------------------------------------------+
| Samples Removed: length outside of expected range |
+-----------------------------+---------------------+
|          Sample ID          |        Length       |
+-----------------------------+---------------------+
|              53             |        536000       |
|              54             |        536000       |
|              55             |        536000       |
+-----------------------------+---------------------+
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [34]:
namedic=db.names(name_cols=['nanotag', 'spike', 'spike2'])

names=[]
for key in namedic:
    res=namedic[key]
    names.append((', ').join(list(res)))

In [42]:
unique_count(names)

{'both, no, 0': 27,
 'nba, no, 0': 1,
 'no, no, 0': 4,
 'NaN, no, 0': 1,
 'both, yes, 0': 27,
 'both, yes, yes': 15}

In [ ]:
#Find NaN values
nan_cols=ys.isna().any() #Find columns with NaN values
nan_rows=ys.loc[:,nan_cols].isna().any(axis=1) #Find the rws with NaN values (in columns with NaN values)
nan_rows.index #Get indexes of rows (= sample ID)

#Drop rows with more than 5 NaN values
ys=ys.dropna(thresh=ys.shape[1]-5)

In [120]:
#Map namedic to sample ID
tag=ys.index.map(lambda x: namedic[x][0])
# names

<h1> Analysis

In [27]:
# pio.renderers.default = "svg"
var=multivar(ys.T,x,names,names)
# figs=var.pca_fig('PC 1','PC 2',ys.index)
# figs=var.pca_figs('PC 2', 'PC 3')
# var.pca_fig(figs)
# var.pca_fig
var.pca_3d(color=names)

In [ ]:
len(db.all_ids)
len(db.dead_list)

54

In [30]:
names

NameError: name 'names' is not defined

<h1>Update Database Find and Replace

In [31]:
reload(data_wrangle)
db=data_wrangle.DataTable('/mnt/c/Users/16162/Desktop/Brain.db','sheep')
db.condition='WHERE sheep IS 2'
db.data_info('nanotag') #Add 'condition' to find_replace
# db.find_replace('BoooTh', 'Booo', 'nanotag')

Number of samples selected: 75


[+-------------------+
 |      nanotag      |
 +-------+-----------+
 | Value | Frequency |
 +-------+-----------+
 |  Booo |     69    |
 |  NaN  |     1     |
 |  nba  |     1     |
 |   no  |     4     |
 +-------+-----------+]